# Mock QCNN Training

In [1]:
from pennylane import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import sys
sys.path.insert(0, '/Users/tak/Github/QEmbedding/')
import Hybrid_nn
import torch
from torch import nn
import data
import pennylane as qml
import embedding

In [2]:
import pennylane as qml
from pennylane import numpy as np
from qiskit import IBMQ
from qiskit.providers.fake_provider import FakeJakarta, FakeMontreal
from qiskit_aer.noise import NoiseModel


noisy = FakeJakarta()
noise_model = NoiseModel.from_backend(noisy)
coupling_map = noisy.configuration().coupling_map
basis_gates = noise_model.basis_gates

dev = qml.device(
    'qiskit.aer',
    wires=4,
    shots=1024,
    noise_model=noise_model,
    coupling_map=coupling_map,
    basis_gates=basis_gates
)

PATH_Model2 = '/Users/tak/Github/QEmbedding/Results/QCNN_demonstration/Real device/Mock/Model2.pt'
PATH_Model3 = '/Users/tak/Github/QEmbedding/Results/QCNN_demonstration/Real device/Mock/Model3.pt'


# Mock QCNN

Hyperparameters

In [3]:
steps = 100
learning_rate = 0.1
batch_size = 10
ansatz = 'TTN'

QCNN

In [4]:
class x_transform2(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.linear_relu_stack2 = nn.Sequential(
            nn.Linear(4, 12),
            nn.ReLU(),
            nn.Linear(12, 12),
            nn.ReLU(),
            nn.Linear(12, 7)
        )
        
    def forward(self, x):
        x = self.linear_relu_stack2(x)
        return x.detach().numpy()


class x_transform3(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.layer1 = torch.nn.Sequential(
            torch.nn.Conv2d(1, 1, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2)
        )

        # Layer2: 14 * 14 -> 7 * 7
        self.layer2 = torch.nn.Sequential(
            torch.nn.Conv2d(1, 1, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2)
        )

        # Fully connected Layers 7 * 7 -> 7
        self.fc = torch.nn.Linear(7 * 7, 7, bias=True)
        
    def forward(self, x):
        x = self.layer1(x)
        x = self.layer2(x)
        x = x.view(-1, 7 * 7)
        x = self.fc(x)
        return x.detach().numpy()

model2 = x_transform2()
model3 = x_transform3()

In [5]:
def statepreparation(x, Trained):
    if Trained == False:
        embedding.Noisy_Four_QuantumEmbedding1(x)
    elif Trained == 'Model2':
        model2.load_state_dict(torch.load(PATH_Model2))
        x = model2(torch.tensor(x))
        embedding.Noisy_Four_QuantumEmbedding2(x)
    elif Trained == 'Model3':
        model3.load_state_dict(torch.load(PATH_Model3))
        x = model3(torch.tensor(x))
        embedding.Noisy_Four_QuantumEmbedding2(x[0])
    

@qml.qnode(dev)
def Noisy_QCNN_classifier(params, x, Trained):
    param1 = params[0:2]
    param2 = params[2:4]

    statepreparation(x, Trained)
    embedding.U_TTN(param1, wires=[0,1])
    embedding.U_TTN(param1, wires=[3,2])
    embedding.U_TTN(param2, wires=[1,2])
    return qml.expval(qml.PauliZ(2))


def Linear_Loss(labels, predictions):
    loss = 0
    for l,p in zip(labels, predictions):
        loss += 0.5 * (1 - l * p)
    return loss / len(labels)


def cost(weights, X_batch, Y_batch, Trained):
    preds = [Noisy_QCNN_classifier(weights, x, Trained) for x in X_batch]
    return Linear_Loss(Y_batch, preds)


def circuit_training(X_train, Y_train, Trained):

    weights = np.random.random(4, requires_grad = True)
    opt = qml.NesterovMomentumOptimizer(stepsize=learning_rate)
    loss_history = []
    for it in range(steps):
        batch_index = np.random.randint(0, len(X_train), (batch_size,))
        X_batch = [X_train[i] for i in batch_index]
        Y_batch = [Y_train[i] for i in batch_index]
        weights, cost_new = opt.step_and_cost(lambda v: cost(v, X_batch, Y_batch, Trained),
                                                     weights)
        loss_history.append(cost_new)
        print("iteration: ", it, " cost: ", cost_new)
    return loss_history, weights

# Model 2

In [6]:

feature_reduction = 'PCA4'
classes = [0,1]
X_train, X_test, Y_train, Y_test = data.data_load_and_process('mnist', feature_reduction=feature_reduction, classes=classes)
X_train, X_test = torch.tensor(X_train).to(torch.float32), torch.tensor(X_test).to(torch.float32)
Y_train = [-1 if y == 0 else 1 for y in Y_train]
Y_test = [-1 if y == 0 else 1 for y in Y_test]

/var/folders/ym/ry24dhs911s7zbqfbhs5ns2m0000gn/T/ipykernel_10345/461388107.py:4: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /Users/runner/work/_temp/anaconda/conda-bld/pytorch_1670525498485/work/torch/csrc/utils/tensor_new.cpp:233.)
  X_train, X_test = torch.tensor(X_train).to(torch.float32), torch.tensor(X_test).to(torch.float32)


In [7]:

Loss_histories_not_trained, weights_not_trained, Loss_histories_Model2, weights_Model2 = [], [], [], []

for i in range(3):
    loss_not_trained, weight_not_trained = circuit_training(X_train, Y_train, False)
    loss_Model2, weight_Model2 = circuit_training(X_train, Y_train, 'Model2')

    Loss_histories_not_trained.append(loss_not_trained)
    Loss_histories_Model2.append(loss_Model2)

    weights_not_trained.append(weight_not_trained)
    weights_Model2.append(weight_Model2)

Loss_histories_not_traiend, Loss_histories_Model2 =  np.array(Loss_histories_not_trained), np.array(Loss_histories_Model2)


f = open('/Users/tak/Github/QEmbedding/Results/QCNN_demonstration/Real device/Mock/Loss_not_trained.txt', 'a')
f.write(str(Loss_histories_not_trained))
f.close()

f = open('/Users/tak/Github/QEmbedding/Results/QCNN_demonstration/Real device/Mock/weights_not_trained.txt', 'a')
f.write(str(weights_not_trained))
f.close()


f = open('/Users/tak/Github/QEmbedding/Results/QCNN_demonstration/Real device/Mock/Loss_Model2.txt', 'a')
f.write(str(Loss_histories_Model2))
f.close()

f = open('/Users/tak/Github/QEmbedding/Results/QCNN_demonstration/Real device/Mock/weights_Model2.txt', 'a')
f.write(str(weights_Model2))
f.close()

/Users/tak/miniforge3/envs/QC/lib/python3.10/site-packages/pennylane/math/utils.py:223: UserWarning: Contains tensors of types {'torch', 'autograd'}; dispatch will prioritize TensorFlow, PyTorch, and  Jax over Autograd. Consider replacing Autograd with vanilla NumPy.
  warnings.warn(
/Users/tak/miniforge3/envs/QC/lib/python3.10/site-packages/autograd/tracer.py:14: UserWarning: Output seems independent of input.
  warnings.warn("Output seems independent of input.")


iteration:  0  cost:  0.50302734375
iteration:  1  cost:  0.4853515625
iteration:  2  cost:  0.503125
iteration:  3  cost:  0.491796875
iteration:  4  cost:  0.50908203125
iteration:  5  cost:  0.50849609375
iteration:  6  cost:  0.4912109375
iteration:  7  cost:  0.47685546875
iteration:  8  cost:  0.50146484375
iteration:  9  cost:  0.4943359375
iteration:  10  cost:  0.46240234375
iteration:  11  cost:  0.4712890625
iteration:  12  cost:  0.4734375
iteration:  13  cost:  0.46884765625
iteration:  14  cost:  0.5244140625
iteration:  15  cost:  0.460546875
iteration:  16  cost:  0.49130859375
iteration:  17  cost:  0.5052734375
iteration:  18  cost:  0.48935546875
iteration:  19  cost:  0.48876953125
iteration:  20  cost:  0.47802734375
iteration:  21  cost:  0.49345703125
iteration:  22  cost:  0.4673828125
iteration:  23  cost:  0.50830078125
iteration:  24  cost:  0.52197265625
iteration:  25  cost:  0.47109375
iteration:  26  cost:  0.46796875
iteration:  27  cost:  0.47978515625


/var/folders/ym/ry24dhs911s7zbqfbhs5ns2m0000gn/T/ipykernel_10345/3053866874.py:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x = model2(torch.tensor(x))


iteration:  0  cost:  0.52216796875
iteration:  1  cost:  0.50693359375
iteration:  2  cost:  0.498046875
iteration:  3  cost:  0.47265625
iteration:  4  cost:  0.48115234375
iteration:  5  cost:  0.4919921875
iteration:  6  cost:  0.4875
iteration:  7  cost:  0.494140625
iteration:  8  cost:  0.48466796875
iteration:  9  cost:  0.48037109375
iteration:  10  cost:  0.45400390625
iteration:  11  cost:  0.473828125
iteration:  12  cost:  0.43388671875
iteration:  13  cost:  0.447265625
iteration:  14  cost:  0.42490234375
iteration:  15  cost:  0.423046875
iteration:  16  cost:  0.429296875
iteration:  17  cost:  0.39453125
iteration:  18  cost:  0.3853515625
iteration:  19  cost:  0.3826171875
iteration:  20  cost:  0.36494140625
iteration:  21  cost:  0.3669921875
iteration:  22  cost:  0.390625
iteration:  23  cost:  0.38369140625
iteration:  24  cost:  0.34384765625
iteration:  25  cost:  0.349609375
iteration:  26  cost:  0.34609375
iteration:  27  cost:  0.34384765625
iteration:  2

# Model3

## 0. Getting Started

In [8]:
feature_reduction = False
X_train, X_test, Y_train, Y_test = data.data_load_and_process('mnist', feature_reduction=feature_reduction, classes=classes)

In [9]:
X_train, X_test = torch.tensor(X_train).to(torch.float32), torch.tensor(X_test).to(torch.float32)
X_train, X_test = X_train.permute(0, 3, 1, 2), X_test.permute(0, 3, 1, 2)
Y_train = [-1 if y == 0 else 1 for y in Y_train]
Y_test = [-1 if y == 0 else 1 for y in Y_test]

In [10]:
Loss_histories_Model3, weights_Model3 = [], []
for i in range(3):

    loss_Model3, weight_Model3 = circuit_training(X_train, Y_train, 'Model3')

    Loss_histories_Model3.append(loss_Model3)
    weights_Model3.append(weight_Model3)

f = open('/Users/tak/Github/QEmbedding/Results/QCNN_demonstration/Real device/Mock/Loss_Model3.txt', 'a')
f.write(str(Loss_histories_Model3))
f.close()

f = open('/Users/tak/Github/QEmbedding/Results/QCNN_demonstration/Real device/Mock/weights_Model3.txt', 'a')
f.write(str(weights_Model3))
f.close()

/var/folders/ym/ry24dhs911s7zbqfbhs5ns2m0000gn/T/ipykernel_10345/3053866874.py:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x = model3(torch.tensor(x))


iteration:  0  cost:  0.475390625
iteration:  1  cost:  0.47900390625
iteration:  2  cost:  0.47353515625
iteration:  3  cost:  0.48447265625
iteration:  4  cost:  0.46025390625
iteration:  5  cost:  0.46982421875
iteration:  6  cost:  0.44677734375
iteration:  7  cost:  0.42578125
iteration:  8  cost:  0.4083984375
iteration:  9  cost:  0.44140625
iteration:  10  cost:  0.4123046875
iteration:  11  cost:  0.41943359375
iteration:  12  cost:  0.3111328125
iteration:  13  cost:  0.3052734375
iteration:  14  cost:  0.33798828125
iteration:  15  cost:  0.27548828125
iteration:  16  cost:  0.39287109375
iteration:  17  cost:  0.2552734375
iteration:  18  cost:  0.2302734375
iteration:  19  cost:  0.27685546875
iteration:  20  cost:  0.25390625
iteration:  21  cost:  0.276953125
iteration:  22  cost:  0.2744140625
iteration:  23  cost:  0.27666015625
iteration:  24  cost:  0.3998046875
iteration:  25  cost:  0.3404296875
iteration:  26  cost:  0.39560546875
iteration:  27  cost:  0.30722656